This notebook illustrates the agent creation process for the **LLM 20 Questions**. Running this notebook produces a `submission.tar.gz` file. You may submit this file directly from the **Submit to competition** heading to the right. Alternatively, from the notebook viewer, click the *Output* tab then find and download `submission.tar.gz`. Click **Submit Agent** at the upper-left of the competition homepage to upload your file and make your submission. 

In [ ]:
%%bash
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
# %%writefile submission/main.py
# Setup
import os
import sys

# **IMPORTANT:** Set up your system path like this to make your code work
# both in notebooks and in the simulations environment.
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

#################################################################

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# Prompt Formatting
import itertools
from typing import Iterable




class GemmaFormatter:
    '''
        A formatter for the Gemma model that allows for easy construction of prompts,
        basically use __repr__ function and other functions to format the prompt

        containing following members:
            self._system_prompt: str
            self._few_shot_examples: Iterable
            self._turn_user: str
            self._turn_model: str
            self._state: str
            self._start_token: str
            self._end_token: str
            self.user: function
            self.model: function
    '''

    
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        '''
            repr funcion is used to return the string representation of the object
            here we retrun the state of the object
        '''
        return self._state

    def user(self, prompt):
        '''

        '''
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        '''
            in this function self._state is reset to empty string and will be modified by user() and model() functions
        '''
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self


    def apply_turns(self, turns: Iterable, start_agent: str):
        '''
        Apply a sequence of turns to the formatter, starting with the specified agent.
        '''
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model] # here, self.model and self.user are functions definded above
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


# Agent Definitions
import re


@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """
    Context Manager
        setting the default torch dtype to the given dtype during this context
    """
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)


class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        # load the configuration of model
        print("Initializing model")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant


        # load the model
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        '''
        *important function
            this function is called when the agent is called in agent_fn() function, which means what happen all start from here
        '''
        self._start_session(obs)
        # format the prompt
        prompt = str(self.formatter)    # here it will call the __repr__ function of the formatter
        # call the LLM to get response
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        '''
            bascially rewrited in questioner and answerer agents
        '''
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        '''
            Parse the keyword from the response.
        '''
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        '''
            Parse the response from the model.
        '''
        raise NotImplementedError


def interleave_unequal(x, y):
    '''
        Interleave two lists of unequal length.
    '''
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        '''
            could be a good example of how to start a session and apply the prompt and etc.
        '''
        self.formatter.reset()
        self.formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("Please ask a yes-or-no question.")
        elif obs.turnType == 'guess':
            self.formatter.user("Now guess the keyword. Surround your guess with double asterisks.")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "Is it a person?"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("Unknown turn type:", obs.turnType)


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"Let's play 20 Questions. You are playing the role of the Answerer. The keyword is {obs.keyword} in the category {obs.category}.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"The question is about the keyword {obs.keyword} in the category {obs.category}. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'


# Agent Creation
system_prompt = "You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."

few_shot_examples = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is it a person?", "**no**",
    "Is is a place?", "**yes**",
    "Is it a country?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]


# **IMPORTANT:** Define agent as a global so you only have to load
# the agent you need. Loading both will likely lead to OOM.
agent = None


def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "Agent not initialized."

    return agent


def agent_fn(obs, cfg):
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    if response is None or len(response) <= 1:
        return "yes"
    else:
        return response

In [ ]:
!apt install pigz pv > /dev/null

In [ ]:
# zip the submission with pigz and pv, where -C means change to the directory before adding the files
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2

In [2]:
# Prompt Formatting
import itertools
from typing import Iterable


class GemmaFormatter:
    '''
        A formatter for the Gemma model that allows for easy construction of prompts,
        basically use __repr__ function and other functions to format the prompt

        containing following members:
            self._system_prompt: str
            self._few_shot_examples: Iterable
            self._turn_user: str
            self._turn_model: str
            self._state: str
            self._start_token: str
            self._end_token: str
            self.user: function
            self.model: function
    '''

    
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        '''
            repr funcion is used to return the string representation of the object
            here we retrun the state of the object
        '''
        return self._state

    def user(self, prompt):
        '''

        '''
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        '''
            in this function self._state is reset to empty string and will be modified by user() and model() functions
        '''
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self


    def apply_turns(self, turns: Iterable, start_agent: str):
        '''
        Apply a sequence of turns to the formatter, starting with the specified agent.
        '''
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model] # here, self.model and self.user are functions definded above
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


system_prompt = "You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."

few_shot_examples = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is it a person?", "**no**",
    "Is is a place?", "**yes**",
    "Is it a country?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]

formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)


In [4]:

def interleave_unequal(x, y):
    '''
        Interleave two lists of unequal length.
    '''
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


#### How to use formatter

basically what formatter has done are just 
1. Adding existing Q&A
2. Putting the user prompt to the last user round 
3. Adding the starting token for model. 
4. Then everything will be sent to the model to generate following sentence

In [9]:
formatter.reset()
print('<== [self.reset()] ==> \n', str(formatter), '\n')
formatter.user(f"Let's play 20 Questions. You are playing the role of the Answerer. The keyword is {'dump_keyword'} in the category {'dump_cate'}.")
print('<== [self.user(user_sys_prompt)] ==> \n', str(formatter), '\n')
turns = interleave_unequal(['Q_1', 'Q_2', 'Q_3'], ['A_1', 'A_2', 'A_3', 'A_4'])
print('<== [interleave_unequal(x, y)] ==> \n', turns, '\n')
formatter.apply_turns(turns, start_agent='user')
print('<== [self.apply_turns( turns, "user")] ==> \n', str(formatter), '\n')
formatter.user(f"The question is about the keyword {'dump_keyword'} in the category {'dump_cate'}. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.")
print('<== [self.user(user_sys_prompt)] ==> \n', str(formatter), '\n')
formatter.start_model_turn()
print('<== [self.start_model_turn()] ==> \n', str(formatter), '\n')

<== [self.reset()] ==> 
 <start_of_turn>user
You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing.<end_of_turn>
<start_of_turn>user
Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.<end_of_turn>
<start_of_turn>model
Is it a person?<end_of_turn>
<start_of_turn>user
**no**<end_of_turn>
<start_of_turn>model
Is is a place?<end_of_turn>
<start_of_turn>user
**yes**<end_of_turn>
<start_of_turn>model
Is it a country?<end_of_turn>
<start_of_turn>user
**yes** Now guess the keyword.<end_of_turn>
<start_of_turn>model
**France**<end_of_turn>
<start_of_turn>user
Correct!<end_of_turn>
 

<== [self.user(user_sys_prompt)] ==> 
 <start_of_turn>user
You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions 